## Seaborn을 이용한 선형 회귀 및 데이터 시각화
- 이번 시간에는 영화 예산과 수익에 대한 자료를 들여다 본다. 이런 DataSet는 "Scikit-Learn"과 같은 툴을 통해 선형 회귀 분석을 실행하고, "Matplotlib"을 기반으로 구축된 데이터 시각화 라이브러리인 "Seaborn"을 사용하기에 매우 적합하다.

<img src="asset/movies.png">

- 이번 시간에 우리가 답할 질문은, 더 많은 영화 예산은 박스 오피스에서 더 많은 수익으로 이어지는지에 대한 것이다.
- 다른 말로, "영화 스튜디오는 더 많은 영화 제작을 위해 영화에 더 많은 돈을 써야하는 것인가?" 이다.

### 이번 시간 Study 내용 
1. "Seaborn"이라는, 유명한 데이터 시각화 라이브러리 사용 방법

<img src="asset/seaborn.png">

2. "Scikit-Learn"을 이용해 선형 회귀 분석을 실행하고 해석

<img src="asset/sklearn.png">

3. 데이터간의 관계를 시각화하는 산점도를 통해, 회귀 분석을 표시하는 방법

3. 산점도에 세 번째 차원을 추가하여 거품 차트를 만드는 방법

3. 내림 나눗셈(//)을 영리하게 사용해, 데이터를 변환하는 방법

### 1. 데이터 탐색 및 정리하기 

In [1]:
# 필요한 패키지 임포트 
import pandas as pd

In [2]:
# 데이터 불러오기 
df = pd.read_csv('data/cost_revenue_dirty.csv')
print(f'Data shape is {df.shape}')
df

Data shape is (5391, 6)


,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
0,5293,8/2/1915,The Birth of a Nation,"$110,000","$11,000,000","$10,000,000"
1,5140,5/9/1916,Intolerance,"$385,907",$0,$0
2,5230,12/24/1916,"20,000 Leagues Under the Sea","$200,000","$8,000,000","$8,000,000"
3,5299,9/17/1920,Over the Hill to the Poorhouse,"$100,000","$3,000,000","$3,000,000"
4,5222,1/1/1925,The Big Parade,"$245,000","$22,000,000","$11,000,000"
...,...,...,...,...,...,...
5386,2950,10/8/2018,Meg,"$15,000,000",$0,$0
5387,126,12/18/2018,Aquaman,"$160,000,000",$0,$0
5388,96,12/31/2020,Singularity,"$175,000,000",$0,$0
5389,1119,12/31/2020,Hannibal the Conqueror,"$50,000,000",$0,$0


In [3]:
# NaN값 확인
df.isnull().sum()
# or df.isna().values.any()

Rank                     0
Release_Date             0
Movie_Title              0
USD_Production_Budget    0
USD_Worldwide_Gross      0
USD_Domestic_Gross       0
dtype: int64

In [4]:
# 중복값 확인 
print(df.duplicated(keep=False).sum())
# or data.duplicated().values.any()

0


In [5]:
# 부분 집합을 만들고 집합의 길이 확인 - 중복 수 확인
duplicated_rows = df[df.duplicated()]
len(duplicated_rows)

0

In [6]:
# 각 열의 데이터 유형 확인 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5391 entries, 0 to 5390
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Rank                   5391 non-null   int64 
 1   Release_Date           5391 non-null   object
 2   Movie_Title            5391 non-null   object
 3   USD_Production_Budget  5391 non-null   object
 4   USD_Worldwide_Gross    5391 non-null   object
 5   USD_Domestic_Gross     5391 non-null   object
dtypes: int64(1), object(5)
memory usage: 252.8+ KB


In [7]:
# 예산 및 총 수익 데이터 각종 특수문자 제거 및 정수형으로 데이터 변환 
df['USD_Production_Budget'] = df['USD_Production_Budget'].replace('[\$,]', '', regex=True).astype(int)
df['USD_Worldwide_Gross'] = df['USD_Worldwide_Gross'].replace('[\$,]', '', regex=True).astype(int)
df['USD_Domestic_Gross'] = df['USD_Domestic_Gross'].replace('[\$,]', '', regex=True).astype(int)

In [8]:
# Release_Date, datetime type으로 변환 
df['Release_Date'] = pd.to_datetime(df['Release_Date'])

In [9]:
# 위를 한번에 처리하기 위한 코드
chars_to_remove = [',', '$']
columns_to_clean = ['USD_Production_Budget', 
                    'USD_Worldwide_Gross',
                    'USD_Domestic_Gross']
 
for col in columns_to_clean:
    for char in chars_to_remove:
        # Replace each character with an empty string
        df[col] = df[col].astype(str).str.replace(char, "")
    # Convert column to a numeric data type
    df[col] = pd.to_numeric(df[col])

/var/folders/6t/12w79lnx68sczzc5f6xs76z80000gn/T/ipykernel_83757/3239217400.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].astype(str).str.replace(char, "")


In [10]:
df

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
0,5293,1915-08-02,The Birth of a Nation,110000,11000000,10000000
1,5140,1916-05-09,Intolerance,385907,0,0
2,5230,1916-12-24,"20,000 Leagues Under the Sea",200000,8000000,8000000
3,5299,1920-09-17,Over the Hill to the Poorhouse,100000,3000000,3000000
4,5222,1925-01-01,The Big Parade,245000,22000000,11000000
...,...,...,...,...,...,...
5386,2950,2018-10-08,Meg,15000000,0,0
5387,126,2018-12-18,Aquaman,160000000,0,0
5388,96,2020-12-31,Singularity,175000000,0,0
5389,1119,2020-12-31,Hannibal the Conqueror,50000000,0,0


In [11]:
# 데이터 전처리 후 데이터 타입 확인 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5391 entries, 0 to 5390
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Rank                   5391 non-null   int64         
 1   Release_Date           5391 non-null   datetime64[ns]
 2   Movie_Title            5391 non-null   object        
 3   USD_Production_Budget  5391 non-null   int64         
 4   USD_Worldwide_Gross    5391 non-null   int64         
 5   USD_Domestic_Gross     5391 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 252.8+ KB


- 데이터들을 분석에 맞게 전처리 작업을 진행하였다.
- 여기까지 확인 결과 데이터는 총 행 5,391개, 열 6개로 조회 되었다.
- NaN과 중복값은 확인되지 않았다. 
- 각 열의 정보와 데이터 유형은 아래와 같다. 
    - Rank(int): 영화 순위
    - Release_Date(datetime): 영화 개봉 날짜 
    - Movie_Title(str): 영화 제목 
    - USD_Production_Budget(int): 영화 예산 (USD 달러)
    - USD_Worldwide_Gross(int): 전세계 수입 총액 (USD 달러)
    - USD_Domestic_Gross(int): 국내 수입 총액 (USD 달러)

### 2. 수익이 "0"인 영화 조사하기 

In [17]:
# 데이터 통계 내역 확인
df.describe()

,Rank,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
count,5391.000000,5.391000e+03,5.391000e+03,5.391000e+03
mean,2696.000000,3.111374e+07,8.885542e+07,4.123552e+07
std,1556.391981,4.052380e+07,1.684578e+08,6.602935e+07
min,1.000000,1.100000e+03,0.000000e+00,0.000000e+00
25%,1348.500000,5.000000e+06,3.865206e+06,1.330902e+06
50%,2696.000000,1.700000e+07,2.745045e+07,1.719220e+07
75%,4043.500000,4.000000e+07,9.645446e+07,5.234369e+07
max,5391.000000,4.250000e+08,2.783919e+09,9.366622e+08


In [24]:
# 영화들의 평균 제작 예산 
production_budget_films = round(df['USD_Production_Budget'].mean(), 2)
print(f'The average production budget of films: ${production_budget_films}')

The average production budget of films: $31113737.58


In [27]:
# 전세계 최소 매출 
min_world_gross = df['USD_Worldwide_Gross'].min()
# 국내 최소 매출
min_domestic_gross = df['USD_Domestic_Gross'].min()

print(f'Minimum worldwide sales is ${min_world_gross}')
print(f'Minimum Domestic sales is ${min_domestic_gross}')

Minimum worldwide sales is $0
Minimum Domestic sales is $0


In [29]:
# 전세계 최대 매출 
max_world_gross = round(df['USD_Worldwide_Gross'].max(), 2)
# 국내 최대 매출
max_domestic_gross = round(df['USD_Domestic_Gross'].max(), 2)

print(f'Maximum worldwide sales is ${max_world_gross}')
print(f'Maximum minimum sales is ${max_domestic_gross}')

Maximum worldwide sales is $2783918982
Maximum minimum sales is $936662225


- 평균적인 영화 제작비는 약 "3,100만 달러"이며 전 세계 수익의 3배(또는 약 8,900만 달러)를 벌어들인다.
- 하지만 많은 영화들이 돈을 날리기도 한다. 하위 4분위에 있는 모든 영화들은 평균 예산(비용)이 5백만 달러지만 전 세계적으로 380만 달러의 수입만을 올리기 때문에 손해를 본다. 
- 국내 및 전 세계 최소 수익은 "0 달러" 이다.
- 반면, 최고 생산 예산은 "4억 2,500만 달러"였고, 전 세계적으로 가장 높은 수익은 "27억 달러"였다.

In [30]:
# 예산이 적게 드는 영화 조회 
df[df.USD_Production_Budget == 1100]

,Rank,Release_Date,Movie_Title,USD_Production_Budget,USD_Worldwide_Gross,USD_Domestic_Gross
2427,5391,2005-05-08,My Date With Drew,1100,181041,181041


- 예산이 적게드는 영화는 "My Date With Drew"이다. 하지만 전세계 수익 및 국내 수익 금액일 보니 예산 대비 돈을 많이 버는 것으로 나타났다.